In [ ]:
from bioservices.uniprot import UniProt #imports uniprot 
import pandas as pd 

In [39]:
Input = '/Users/ken/Box/proj/proXtal/justin_redoxPotential/data/RISE2020/data/2020_Raanan/key_value.csv'
key_value = pd.read_table(Input,sep=",",header=0) # import the key_value.csv file
print(key_value.columns)
# print(key_value)

Index(['SP_PRIMARY', 'pdb_chain', 'ID', 'module', 'cofactor_id',
       'cofactor_type', 'ec', 'name', 'species', 'superkingdom', 'pdbID',
       'chainID', 'residueNum', 'cofactor_id2'],
      dtype='object')


In [41]:
import requests
from xml.etree.ElementTree import fromstring

# pdb_id = '4hhb.A' #example
pdb_mapping_url = 'http://www.rcsb.org/pdb/rest/das/pdb_uniprot_mapping/alignment'
uniprot_url = 'http://www.uniprot.org/uniprot/{}.xml'

def get_uniprot_accession_id(response_xml):
    root = fromstring(response_xml)
    return next(
        el for el in root.getchildren()[0].getchildren()
        if el.attrib['dbSource'] == 'UniProt'
    ).attrib['dbAccessionId']

def get_uniprot_protein_name(uniport_id):
    uinprot_response = requests.get(
        uniprot_url.format(uniport_id)
    ).text
    return fromstring(uinprot_response).find(
        './/{http://uniprot.org/uniprot}recommendedName/{http://uniprot.org/uniprot}fullName'
    ).text

def map_pdb_to_uniprot(pdb_id):
    pdb_mapping_response = requests.get(
        pdb_mapping_url, params={'query': pdb_id}
    ).text
    uniprot_id = get_uniprot_accession_id(pdb_mapping_response)
    uniprot_name = get_uniprot_protein_name(uniprot_id)
    return {
        'pdb_id': pdb_id,
        'uniprot_id': uniprot_id,
        'uniprot_name': uniprot_name
    }
for i, row in key_value.iterrows():
    # print(row.pdbID)
    pdb_id = row.pdbID+"."+row.chainID
    print(map_pdb_to_uniprot(pdb_id))

NameError: name 'pdb_id' is not defined

In [ ]:
u = UniProt(verbose=True)
for i, row in key_value.iterrows():
    # print(row.pdbID)
    map = u.mapping("PDB_ID","ACC", row.pdbID)
    # print(map)
    if(len(map[row.pdbID])>1):

        for ac in map[row.pdbID]:
            print(map)
            print(ac)
            fasta = u.get_fasta_sequence(ac)
            print(fasta)